In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from concurrent.futures import ThreadPoolExecutor
import os
from tqdm import tqdm
from sklearn.impute import KNNImputer
from sklearn.base import clone
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, accuracy_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [47]:
# Constants
SEED = 42

In [48]:
# Loading the datasets
train_file_path = "train_ft.csv"
train = pd.read_csv(train_file_path)
test_file_path = "test_ft.csv"
test = pd.read_csv(test_file_path)
df = train

In [49]:
train.dropna(subset=['sii'], how='all', inplace=True)


In [50]:
'''train[['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',
           'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',
           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',
           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',
           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',
           'PCIAT-PCIAT_20']] = train[['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',
           'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',
           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',
           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',
           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',
           'PCIAT-PCIAT_20']].fillna(5) # Fill NaN values with 5'''

"train[['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',\n           'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',\n           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',\n           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',\n           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',\n           'PCIAT-PCIAT_20']] = train[['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',\n           'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',\n           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',\n           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',\n           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',\n           'PCIAT-PCIAT_20']].fillna(5) # Fill NaN values with 5"

In [51]:
'''train['PCIAT_Total_Imputed'] = train[['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',
           'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',
           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',
           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',
           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',
           'PCIAT-PCIAT_20']].sum(axis=1)'''

"train['PCIAT_Total_Imputed'] = train[['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',\n           'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',\n           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',\n           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',\n           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',\n           'PCIAT-PCIAT_20']].sum(axis=1)"

In [52]:
sii_thresholds = [0, 30, 50, 80, 100] # Thresholds for rating
def get_rating(x): ## Thresholds for rating
    if 0 <= x <= 29:
        return 0
    elif 30 <= x <= 49:
        return 1
    elif 50 <= x <= 79:
        return 2
    else:
        return 3

def quadratic_weighted_kappa(y_true, y_pred): # The Quadric Kappa Evaluation Metric
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def evaluate_predictions(y_true, oof_non_rounded):
    rounded_p = get_rating(oof_non_rounded)
    return -quadratic_weighted_kappa(y_true, rounded_p)

#train_impute_lr['new_sii'] = train_impute_lr['PCIAT_Total_Imputed'].apply(get_rating)

In [53]:
train_model = train
X = train_model.drop(columns=['id', 'sii', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03','PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',
           'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',
           'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',
           'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',
           'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total'])
Y = train['PCIAT-PCIAT_Total']
for col in X.columns:
    if X[col].dtype == 'object' or str(X[col].dtype) == 'category':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))

In [54]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []
for train_idx, val_idx in kf.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = Y.iloc[train_idx], Y.iloc[val_idx]

    model = XGBRegressor(n_estimators=100, learning_rate=0.1)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=True
    )

    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    rmse_scores.append(rmse)

print("Average RMSE across folds:", np.mean(rmse_scores))

[0]	validation_0-rmse:20.44325
[1]	validation_0-rmse:20.07227
[2]	validation_0-rmse:19.74152
[3]	validation_0-rmse:19.44684
[4]	validation_0-rmse:19.14312
[5]	validation_0-rmse:18.96125
[6]	validation_0-rmse:18.84557
[7]	validation_0-rmse:18.68123
[8]	validation_0-rmse:18.60034
[9]	validation_0-rmse:18.53428
[10]	validation_0-rmse:18.46875
[11]	validation_0-rmse:18.45318
[12]	validation_0-rmse:18.38242
[13]	validation_0-rmse:18.31569
[14]	validation_0-rmse:18.34262
[15]	validation_0-rmse:18.29923
[16]	validation_0-rmse:18.25345
[17]	validation_0-rmse:18.26573
[18]	validation_0-rmse:18.30668
[19]	validation_0-rmse:18.29696
[20]	validation_0-rmse:18.30844
[21]	validation_0-rmse:18.27362
[22]	validation_0-rmse:18.28224
[23]	validation_0-rmse:18.25971
[24]	validation_0-rmse:18.23312
[25]	validation_0-rmse:18.22671
[26]	validation_0-rmse:18.21775
[27]	validation_0-rmse:18.18738
[28]	validation_0-rmse:18.20600
[29]	validation_0-rmse:18.19550
[30]	validation_0-rmse:18.21465
[31]	validation_0-

In [55]:
train = train.reset_index(drop=True)

In [56]:
predictions_xgb = model.predict(X)

In [57]:
predictions_xgb.shape

(2736,)

In [58]:
pd.Series(predictions_xgb).isnull().sum()

0

In [59]:
train['predicted_val'] = pd.Series(predictions_xgb).apply(get_rating)

In [60]:
train['predicted_val']

0       1
1       0
2       1
3       1
4       0
       ..
2731    0
2732    0
2733    1
2734    1
2735    0
Name: predicted_val, Length: 2736, dtype: int64

In [61]:
quadratic_weighted_kappa(train['sii'], train['predicted_val'])

0.6460869312921528

In [62]:
train.to_csv("final_values.csv", index=False)

In [63]:
misclassification_error = 1 - accuracy_score(train['sii'], train['predicted_val'])
print("Misclassification Error:", misclassification_error)

Misclassification Error: 0.23282163742690054


In [64]:
for col in test.columns:
    if test[col].dtype == 'object' or str(test[col].dtype) == 'category':
        le = LabelEncoder()
        test[col] = le.fit_transform(test[col].astype(str))

In [65]:
X_test = test.drop(columns=['id', 'BIA-BIA_BMI', 'SDS-SDS_Total_T'])

In [66]:
# test_impute_lr = test_impute_lr.drop(columns=['id'])
predictions_xgb_test = model.predict(X_test)

In [67]:
test['predicted_val'] = pd.Series(predictions_xgb_test).apply(get_rating)

In [68]:
test['predicted_val'].isnull().sum()

0

In [69]:
test['predicted_val'].shape

(20,)

In [70]:
test_to_save = test[['id', 'predicted_val']]
test_to_save.rename(columns={'predicted_val': 'sii'}, inplace=True)
test_to_save.to_csv("submission.csv", index=False)

/var/folders/tn/htz4wbqn6g11g465bmx5l4tr0000gn/T/ipykernel_47224/3441375821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_to_save.rename(columns={'predicted_val': 'sii'}, inplace=True)
